In [1]:
import sys
import configparser
import logging
import plotly.graph_objects as go
sys.path.append(r'L:\Python\ALMAFE-Instr')
sys.path.append(r'L:\Python\ALMAFE-AMBDeviceLibrary')
from INSTR.InputSwitch.ExternalSwitch import ExternalSwitch, InputSelect
from INSTR.SpectrumAnalyzer.SpectrumAnalyzer import SpectrumAnalyzer, SpectrumAnalyzerSettings
from Control.IFSystem.TemporaryB6v2 import IFSystem
from Control.PowerDetect.PDSpecAn import PDSpecAn
from Control.RFAutoLevel import RFAutoLevel
from Control.RFSource import RFSource
from AMB.AMBConnectionDLL import AMBConnectionDLL

CARTRIDGE_BAND = 6
RF_SOURCE_PORT = 7
RF_SOURCE_PA_POL = 0
NODE_ADDR = 0x13

LOG_TO_FILE = True
LOG_FILE = 'ALMAFE-CTS-Control.log'

logger = logging.getLogger("ALMAFE-CTS-Control")
logger.setLevel(logging.INFO)
if LOG_TO_FILE:
    handler = logging.FileHandler(LOG_FILE)
else:
    handler = logging.StreamHandler()
handler.setFormatter(logging.Formatter(fmt = '%(asctime)s %(levelname)s:%(message)s'))
logger.addHandler(handler)

logger2 = logging.getLogger("ALMAFE-AMBDeviceLibrary")
logger2.setLevel(logging.INFO)
logger2.addHandler(handler)

logger.info("---- test_AutoLevel start ----")

config = configparser.ConfigParser()
config.read('FrontEndAMBDLL.ini')
dllName = config['load']['dll']
conn = AMBConnectionDLL(channel = RF_SOURCE_PA_POL, dllName = dllName)
rfSource = RFSource(conn, NODE_ADDR, CARTRIDGE_BAND, RF_SOURCE_PORT, RF_SOURCE_PA_POL)

externalSwitch = ExternalSwitch()
spectrumAnalyzer = SpectrumAnalyzer()
ifSystem = IFSystem(externalSwitch, spectrumAnalyzer)
powerDetect = PDSpecAn(spectrumAnalyzer)
rfAutoLevel = RFAutoLevel(ifSystem, powerDetect, rfSource)


In [4]:
specAnSettings = SpectrumAnalyzerSettings(attenuation = 2, resolutionBW = 10e3, enableInternalPreamp = False)
powerDetect.configure(config = specAnSettings)
rfAutoLevel.autoLevel(10, -30)

(True, '')